In [85]:
path = './zomato.csv'

In [157]:
import csv
import os
import glob
import numpy as np
import math
#scikit learn
from sklearn.pipeline import Pipeline
from sklearn.metrics import accuracy_score
from sklearn.model_selection import GridSearchCV

#stemmer
from Sastrawi.Stemmer.StemmerFactory import StemmerFactory
from Sastrawi.StopWordRemover.StopWordRemoverFactory import StopWordRemoverFactory

In [143]:
def cleaning(sentence):
    sentence = sentence.replace('\ufeff','')
    sentence = sentence.replace('\n',' ')
    sentence = sentence.replace('\r',' ')
    sentence = sentence.replace('\n\n',' ')
    sentence = sentence.replace('\r\r',' ')
    sentence = sentence.replace('.',' ')
    sentence = sentence.replace(',',' ')
    sentence = sentence.replace('"',' ')
    sentence = sentence.replace('!',' ')
    sentence = sentence.replace('?',' ')
    sentence = sentence.replace('/',' ')
    sentence = sentence.replace('-',' ')
    sentence = sentence.lstrip()
    return sentence.lower()

In [149]:
def convert(sentences):
    import re
    cleanEmoticon = []
    for sentence in sentences:
        try:
            highpoints = re.compile(u'([\U00002600-\U000027BF])|([\U0001f300-\U0001f64F])|([\U0001f680-\U0001f6FF])')
        except re.error:
            highpoints = re.compile(u'([\u2600-\u27BF])|([\uD83C][\uDF00-\uDFFF])|([\uD83D][\uDC00-\uDE4F])|([\uD83D][\uDE80-\uDEFF])')
    
        cleanEmoticon.append(highpoints.sub(u'\u25FD', sentence))
    return cleanEmoticon

In [88]:
x = []
y = []
with open(path) as a:
    b = csv.reader(a)
    for row in b:
        x.append(cleaning(row[2]))
        y.append(row[3])

In [89]:
randomize = np.arange(len(x))
np.random.shuffle(randomize)

In [90]:
x_train = x[:85]
y_train = y[:85]

x_test = x[85:]
y_test = y[85:]

In [171]:
def preprocess(sentences): 
    factory = StopWordRemoverFactory()
    stopwords = factory.get_stop_words()

    factory = StopWordRemoverFactory()
    stopword = factory.create_stop_word_remover()

    result =[]
    for sentence in sentences:
        result.append(stemmer.stem(stopword.remove(sentence)))
    return result

In [172]:
# stem_test = ['dia biasanya makan nasi padang', 
#              'aku menyukainya dengan tulus dan raisa', 
#              'ku tak kan pernah memilikinya sampai kapanpun', 
#              'karena dia sudah memilikinya & dimilikinya']

x_train = preprocess(x_train)
x_test = preprocess(x_test)

In [173]:
x_train = convert(x_train)
x_test = convert(x_test)

### Scikit-Learn Tweaks
Reference:
https://towardsdatascience.com/machine-learning-nlp-text-classification-using-scikit-learn-python-and-nltk-c52b92a7c73a

In [174]:
from sklearn import svm

text_clf = Pipeline([('vect', CountVectorizer()),
                     ('tfidf', TfidfTransformer()),
                     ('clf', svm.SVC(kernel='linear')),
])
text_clf = text_clf.fit(x_train, y_train)

In [175]:
y_pred = text_clf.predict(x_test)

In [176]:
accuracy_score(y_test, y_pred)

0.8666666666666667

In [178]:
x_train

['parah si ceker parah yg pernah d makan ga sesuai advertising banget laper uda nunggu g k pas buka ceker begini yg dateng sedih banget yaa hitam selera makan hilang trs yg lebih parah kuah loh rasa persis sambel bangkok tp versi kuah and asin banget bener kecewa sih n ga makan bener ilfeel ga beli sihhhh cukup tau aja yaa',
 'isi ceker nya banyak pedes nya juara ceker nya lembut buat gue bingung gimana makan ceker alas gue mau makan ceker samping diajarin sih sama cowo gue gimana cara makan ceker nya buat minum halau galau nya enak asem asem bawa jeruk nipis nya',
 'pas buat tempat meetup sama temen temen because they provide many chairs yang panjang jadi gather around datang sini malam malam jadi sempat bingung cari nyata pas samping pom bensin ruko ruko dekat the majesty apartment makan so so lah cobain kopi sih silly sayang nggak terima debit waktu mau bayar entah rusak gimana so you need to prepare cash kalau sini',
 '1 lokasi mudah temu krn strategis pas pengkol sesuai nama resto

In [155]:
y_test

['0', '0', '0', '0', '0', '1', '0', '0', '0', '0', '1', '0', '0', '0', '0']